# Problemas

Como vimos acima, há muitos passos na criação e definição de uma nova rede neural.
A grande parte desses ajustes dependem diretamente do problemas.

Abaixo, listamos alguns problemas. Todos os problemas e datasets usados vem do [Center for Machine Learning and Intelligent Systems](http://archive.ics.uci.edu/ml/datasets.php).


**Seu objetivo é determinar e implementar um modelo para cada problema.**

Isso inclui:

1. definir uma arquitetura.
Por enquanto usando somente camadas [Lineares](https://pytorch.org/docs/stable/nn.html#linear), porém podemos variar as ativações, como [Sigmoid](https://pytorch.org/docs/stable/nn.html#sigmoid), [Tanh](https://pytorch.org/docs/stable/nn.html#tanh), [ReLU](https://pytorch.org/docs/stable/nn.html#relu), [LeakyReLU](https://pytorch.org/docs/stable/generated/torch.nn.LeakyReLU.html), [ELU](https://pytorch.org/docs/stable/generated/torch.nn.ELU.html), [SeLU](https://pytorch.org/docs/stable/generated/torch.nn.SELU.html), [PReLU](https://pytorch.org/docs/stable/generated/torch.nn.PReLU.html), [RReLU](https://pytorch.org/docs/stable/generated/torch.nn.RReLU.html)
2. definir uma função de custo. Algums opções que vimos previamente incluem[L1](https://pytorch.org/docs/stable/nn.html#l1loss), [L2/MSE](https://pytorch.org/docs/stable/nn.html#mseloss), [Huber/SmoothL1](https://pytorch.org/docs/stable/nn.html#smoothl1loss), [*Cross-Entropy*](https://pytorch.org/docs/stable/nn.html#crossentropyloss), [Hinge](https://pytorch.org/docs/stable/nn.html#hingeembeddingloss)), e
3. definir um algoritmo de otimização ([SGD](https://pytorch.org/docs/stable/optim.html#torch.optim.SGD), [RMSProp](https://pytorch.org/docs/stable/optim.html#torch.optim.RMSprop), [Adam](https://pytorch.org/docs/stable/optim.html#torch.optim.Adam))

A leitura do dado assim como a função de treinamento já estão implementados para você.

# Preâmbulo

In [1]:
# imports basicos
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

import torch
import torch.nn.functional as F
import torchvision

from torchvision import datasets, transforms
from torch import optim, nn

import os
import sys
import time
import numpy as np

In [2]:
import matplotlib.pyplot as plt
plt.ion()

In [3]:
# Test if GPU is avaliable, if not, use cpu instead
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
n = torch.cuda.device_count()
devices_ids = list(range(n))
device

device(type='cuda')

## Funções básicas

Use a função `load_array ` declarada a seguir se voce ja tem os dados armazenados em um **array** (por exemplo um array do numpy, o `np.array`). Pode acontecer de que os nossos dados vêm simplesmente de um dataset que pode ser armazennado em um array, e portanto não é necessário fazer os outros passos mais complicados como carregar os dados do disco, etc; basta que possamos recuperar esses dados em *batches* aleatórios. O resultado dessa função é um `DataLoader` do Pytorch com os dados que fornecemos de entrada, e que permite que os acessamos da seguinte forma:

```python
data_loader = load_array(X, y, batch_size=32, is_train=True)
for x_batch, y_batch in data_loader:
    ### ... nossa iteração de treinamento aqui.
```

Essa função recebe como parâmetro os seguintes valores:

- `features`: um array que contém as features de todas as instâncias do dataset. Por exemplo, no caso do MNIST seria um array de tamanho `(60000, 28, 28, 1)` com todas as imagens do dataset de treino.
- `labels`: um array que contém os rótulos de cada instância de dados. No caso do MNIST, seria um array de tamanho `(60000,)` em que a posição `i` contém o rótulo do dígito da posição `i` do array `features`.
- `batch_size`: tamanho do batch desejado
- `is_train`: um booleano que indica se o dataset que estamos criando é o conjunto de treinamento ou não (conjunto de teste). A única mudança que isso causa no `Dataloader` resultante é que se for o conjunto de treinamento ele cria batches aleatórios.


In [4]:
def load_array(features, labels, batch_size, is_train=True):
    """Construct a Torch data loader"""

    ## transform the input arrays in a tensor in case they are not
    if type(features) != torch.tensor:
        features = torch.tensor(features)
    if type(labels) != torch.tensor:
        labels = torch.tensor(labels)

    ## create a Pytorch Dataset and DataLoader with the input data
    dataset = torch.utils.data.TensorDataset(features, labels)
    return torch.utils.data.DataLoader(dataset, batch_size, shuffle=is_train)

Use a função `evaluate_accuracy` para calcular a acurácia e a *loss-function* para a rede em um conjunto de dados. Note que essa função pode ser usada tanto para avaliar a rede no conjunto de teste (no caso que usamos o `DataLoader` de teste) quanto o conjunto de treinamento (se usamos o `DataLoader` de treinamento). Os parâmetros são:
- `data_iter`: um `DataLoader` que contém os dados que queremos usar para avaliar a rede. Repare que esse parâmetro tipicamente é o ojeto que obtemos como saída da função `load_array` para montar o nosso `DataLoader`.
- `net`: a rede que queremos avaliar
- `loss`: a nossa *loss-function*. Pode ser um objeto de qualquer uma das funções de perda que mencionamos acima no começo do notebook.

O resultado dessa função é uma tupla em que o primeiro valor é a acurácia e o segundo a função de custo calculados.

In [5]:
# Função usada para calcular acurácia
def evaluate_accuracy(data_iter, net, loss):
    """Evaluate accuracy of a model on the given data set."""

    ## valores "acumuladores", que guardam a soma de, respectivamente, quantas instâncias
    ## prevemos corretamente, quantas instâncias percorremos no dataset, e o valor da loss; para
    ## todos os batches
    acc_sum, n, l = 0, 0, 0

    ## muda a rede para o "modo de teste". O que isso faz é mudar o comportamento de alguns módulos da rede,
    ## como os módulos de Dropout e BatchNorm, que funcionam de forma diferente quando estamos treinando ou
    ## quando estamos avaliando (ou usando em produção) a rede
    net.eval()

    with torch.no_grad():
      for X, y in data_iter:
          X, y = X.to(device), y.to(device)
          y_hat = net(X)
          l += loss(y_hat, y.long())

          ## aqui estamos calculando a quantidade de previsões que temos correta para o batch atual. o resultado
          ## do argmax é a posição de `y_hat` que possui o maior valor. Consequentemente isso resulta na classe que
          ## a rede deu o maior score.
          acc_sum += (y_hat.argmax(axis=1) == y).sum().item()

          ##
          n += y.size(0)

    return acc_sum / n, l.item() / len(data_iter)

A função `train_validate` é a função que implementa nossas iterações de treinamento padrão. Ela ja faz o trabalho de percorrer o dataset inteiro para cada época, e também de tempos em tempos avaliar a rede e mostar os resultados na tela. Para isso ela faz chamadas à função `evaluate_accuracy` declarada anteriormente (entre outras coisas). Essa função tem os segugintes parâmetros:
- `net`: a rede que queremos treinar
- `train_iter` e `test_iter`: nossos `DataLoaders` que criamos para acessar os dados. Esses DataLoaders podem ser criados com a função `load_array` declarada acima.
- `trainer`: é o nosso otimizador. Podemos usar aqui qualquer um dos otimizadores que escolhermos da lista citada no começo desse notebook.
- `loss`: a loss function que escolhemos para otimizar. Pode ser qualquer um das funções de custo citadas no começo do notebook.
- `num_epochs`: a quantidade de épocas pelas quais queremos que o treinamento ocorra.
- `type`: o tipo de tarefa que estamos lidando. Se for um problema de regressão, usamos `type='regression'`, e se for um problema de classificação, usamos `type='classification'`. Esse parâmetro é necessário para a função, por exemplo, saber quais métricas ele vai mostrar (acurácia, ou apenas o MSE, etc.)

In [6]:
# Função usada no treinamento e validação da rede
def train_validate(net, train_iter, test_iter, trainer, loss, num_epochs, type='regression'):
    print('training on', device)
    for epoch in range(num_epochs):
        train_l_sum, train_acc_sum, n, start = 0.0, 0.0, 0, time.time()
        for X, y in train_iter:
            X, y = X.to(device), y.to(device)
            trainer.zero_grad()
            y_hat = net(X)
            if type == 'regression':
              l = loss(y_hat, y.float())
            else:
              l = loss(y_hat, y.long())
            l.backward()
            trainer.step()
            train_l_sum += l.item()
            train_acc_sum += (y_hat.argmax(axis=1) == y).sum().item()
            n += y.size(0)
        test_acc, test_loss = evaluate_accuracy(test_iter, net, loss)
        if type == 'regression':
          print('epoch %d, train loss %.4f, test loss %.4f, time %.1f sec'
                % (epoch + 1, train_l_sum / len(train_iter), test_loss, time.time() - start))
        else:
          print('epoch %d, train loss %.4f, train acc %.3f, test loss %.4f, '
              'test acc %.3f, time %.1f sec'
              % (epoch + 1, train_l_sum / len(train_iter), train_acc_sum / n, test_loss,
                 test_acc, time.time() - start))


Use a função a seguir para inicializar os pesos da rede. Ela recebe como parâmetro um módulo da rede neural, e se for uma camada linear ele inicializa os pesos e os bias dessa camada. Embora possa parecer complicado de precisar chamar essa função para todas as camadas lineares da nossa rede, o módulos do Pytorch (que incluem tanto as redes criadas com o `nn.Sequential` ou com `nn.Module`) possuem a função `net.apply()` que recebe como parâmetro uma função e aplica ela a todos os submódulos da rede. Portanto, depois de ter criado a nossa rede, podemos chamar:

```python
net.apply(weights_init)
```
que automaticamente todas as camadas `nn.Linear` serão inicializadas. Caso queira saber mais sobre o `.apply()`, veja o seguinte [link](https://pytorch.org/docs/stable/generated/torch.nn.Module.html#torch.nn.Module.apply).

In [7]:
# Função para inicializar pesos da rede
def weights_init(m):
    if type(m) == nn.Linear:
        m.weight.data.normal_(0.0, 0.01) # valores iniciais são uma normal
        m.bias.data.fill_(0)

# Problema 1

Neste problema, você receberá 14 *features* coletadas de pacientes e tentará predizer se eles tem algum sinal de doença cardíaca. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Heart+Disease

In [85]:
## aqui fazemos o download do dataset usando o `!wget`. Se estamos rodando em um servidor linux (como é o caso do Colab),
## podemos usar comandos do linux precedidos pelo "!". Por exemplo podemos fazer !ls para listar os arquivos da instância do colab.
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data

## aqui fazemos um tratamento inicial dos dados. "np.genfromtxt" lê os dados de um arquivo .txt e transforma em
## um array. Pode ser interessante abrir o arquivo para verificar como os dados chegaram. Se estiver no colab, voce
## pode verificar o arquivo "processed.cleveland.data" clicando na pastinha do canto esquerdo da página. a função
## "np.nan_to_num" trata valores NaN e infinitos no dataset.
data = np.genfromtxt('processed.cleveland.data', delimiter=',', dtype=np.float32)
data = np.nan_to_num(data)

--2025-05-20 01:38:39--  https://archive.ics.uci.edu/ml/machine-learning-databases/heart-disease/processed.cleveland.data
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘processed.cleveland.data.1’

processed.cleveland     [ <=>                ]  18.03K  --.-KB/s    in 0.03s   

2025-05-20 01:38:39 (693 KB/s) - ‘processed.cleveland.data.1’ saved [18461]



In [86]:
## aqui separamos os dados entre features (X) e rótulo (y), e depois separamos em um conjunto de treinamento e teste
print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.30, random_state=42)

(303, 14) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6.    0. ]
(303, 13) [ 63.    1.    1.  145.  233.    1.    2.  150.    0.    2.3   3.    0.
   6. ]
(303,) 0.0


In [87]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [88]:
## aqui criamos nossos DataLoaders para conseguirmos iterar nos dados
batch_size = 32
train_iter = load_array(X_train, Y_train, batch_size)
test_iter = load_array(X_test, Y_test, batch_size, False)


In [89]:
class NET (nn.Module):
  def __init__(self):
    super(NET, self).__init__()

    self.nn = nn.Sequential(

      nn.Linear(13, 16),
      nn.Dropout(0.5),

      nn.Tanh(),


      nn.Linear(16, 12),
      nn.Dropout(0.3),
      nn.Tanh(),

      nn.Linear(12, 8),
      nn.Dropout(0.15),
      nn.Tanh(),





      nn.Linear(8, 5),

    )

  def forward(self, X):
    out = self.nn(X)

    return out

In [90]:
model = NET().to(device)
loss = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=0.03, weight_decay = 0.0001)

In [91]:
train_validate(net = model, train_iter = train_iter , test_iter = test_iter, trainer = opt, loss = loss, num_epochs = 100, type='classification')

training on cuda
epoch 1, train loss 1.4614, train acc 0.420, test loss 1.0424, test acc 0.604, time 0.0 sec
epoch 2, train loss 0.9895, train acc 0.637, test loss 1.0422, test acc 0.571, time 0.0 sec
epoch 3, train loss 0.9269, train acc 0.637, test loss 1.0340, test acc 0.582, time 0.0 sec
epoch 4, train loss 0.8981, train acc 0.646, test loss 1.0679, test acc 0.582, time 0.0 sec
epoch 5, train loss 0.8579, train acc 0.656, test loss 1.0505, test acc 0.549, time 0.0 sec
epoch 6, train loss 0.8325, train acc 0.632, test loss 1.0706, test acc 0.560, time 0.0 sec
epoch 7, train loss 0.7780, train acc 0.665, test loss 1.0893, test acc 0.582, time 0.0 sec
epoch 8, train loss 0.7504, train acc 0.708, test loss 1.0958, test acc 0.560, time 0.0 sec
epoch 9, train loss 0.7052, train acc 0.675, test loss 1.1457, test acc 0.571, time 0.0 sec
epoch 10, train loss 0.6569, train acc 0.712, test loss 1.1770, test acc 0.582, time 0.0 sec
epoch 11, train loss 0.6413, train acc 0.722, test loss 1.1900

In [16]:
evaluate_accuracy(test_iter, model, loss = loss)

(0.5604395604395604, 2.3897581100463867)

# Problema 2

Neste problema, você receberá 90 *features* extraídas de diversas músicas (datadas de 1922 até 2011) e deve predizer o ano de cada música. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/YearPredictionMSD

In [17]:
# download do dataset
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
!unzip YearPredictionMSD.txt.zip
data = np.genfromtxt('YearPredictionMSD.txt', delimiter=',', dtype=np.float32)



--2025-05-20 01:03:33--  http://archive.ics.uci.edu/ml/machine-learning-databases/00203/YearPredictionMSD.txt.zip
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘YearPredictionMSD.txt.zip’

YearPredictionMSD.t     [            <=>     ] 201.24M  88.7MB/s    in 2.3s    

2025-05-20 01:03:36 (88.7 MB/s) - ‘YearPredictionMSD.txt.zip’ saved [211011981]

Archive:  YearPredictionMSD.txt.zip
  inflating: YearPredictionMSD.txt   


In [18]:
print(data[0, :])
X, y = torch.from_numpy(data[:, 1:]),torch.from_numpy(data[:, 0])
X_train, X_test, Y_train, Y_test = train_test_split(X, y, test_size=0.33, random_state=42)

scaler = preprocessing.StandardScaler()

X_train = torch.from_numpy(scaler.fit_transform(X_train)).to(device).float()
X_test = torch.from_numpy(scaler.transform(X_test)).to(device).float()
batch_size = 100
train_iter = load_array(X_train, Y_train, batch_size)
test_iter = load_array( X_test, Y_test, batch_size, False)

[ 2.0010000e+03  4.9943569e+01  2.1471140e+01  7.3077499e+01
  8.7486095e+00 -1.7406281e+01 -1.3099050e+01 -2.5012020e+01
 -1.2232570e+01  7.8308902e+00 -2.4678299e+00  3.3213601e+00
 -2.3152101e+00  1.0205560e+01  6.1110913e+02  9.5108960e+02
  6.9811426e+02  4.0898486e+02  3.8370911e+02  3.2651511e+02
  2.3811327e+02  2.5142413e+02  1.8717351e+02  1.0042652e+02
  1.7919498e+02 -8.4155798e+00 -3.1787039e+02  9.5862663e+01
  4.8102589e+01 -9.5663033e+01 -1.8062149e+01  1.9698400e+00
  3.4424381e+01  1.1726700e+01  1.3679000e+00  7.7944398e+00
 -3.6994001e-01 -1.3367851e+02 -8.3261650e+01 -3.7297649e+01
  7.3046669e+01 -3.7366840e+01 -3.1385300e+00 -2.4215309e+01
 -1.3230660e+01  1.5938090e+01 -1.8604780e+01  8.2154793e+01
  2.4057980e+02 -1.0294070e+01  3.1584311e+01 -2.5381870e+01
 -3.9077201e+00  1.3292580e+01  4.1550598e+01 -7.2627201e+00
 -2.1008631e+01  1.0550848e+02  6.4298561e+01  2.6084810e+01
 -4.4591099e+01 -8.3065701e+00  7.9370599e+00 -1.0736600e+01
 -9.5447662e+01 -8.20330

<ipython-input-4-f6f8c2b9f8ba>:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  features = torch.tensor(features)
<ipython-input-4-f6f8c2b9f8ba>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  labels = torch.tensor(labels)


In [19]:
class NET2 (nn.Module):
  def __init__(self):
    super(NET2, self).__init__()

    self.nn = nn.Sequential(

      nn.Linear(90, 32),
      nn.ReLU(),

      nn.Linear(32, 16),
      nn.ReLU(),

      nn.Linear(16, 8),
      nn.ReLU(),

      nn.Linear(8, 4),
      nn.ReLU(),



      nn.Linear(4, 1),

    )

  def forward(self, X):
    out = self.nn(X)

    return out

In [20]:
model2 = NET2().to(device)
loss = nn.MSELoss()
opt = optim.Adam(model2.parameters(), lr=0.001, weight_decay = 0.001)

In [21]:
train_validate(net = model2, train_iter = train_iter , test_iter = test_iter, trainer = opt, loss = loss, num_epochs = 20, type='regression')

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([100])) that is different to the input size (torch.Size([100, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


training on cuda


/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([81])) that is different to the input size (torch.Size([81, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/usr/local/lib/python3.11/dist-packages/torch/nn/modules/loss.py:610: UserWarning: Using a target size (torch.Size([64])) that is different to the input size (torch.Size([64, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


epoch 1, train loss 425580.0672, test loss 16416.9982, time 12.5 sec
epoch 2, train loss 3452.1928, test loss 405.8616, time 12.3 sec
epoch 3, train loss 231.7885, test loss 210.5230, time 12.3 sec
epoch 4, train loss 163.6173, test loss 163.5506, time 13.4 sec
epoch 5, train loss 156.6720, test loss 185.4185, time 12.4 sec
epoch 6, train loss 151.8878, test loss 137.6621, time 12.3 sec
epoch 7, train loss 147.3531, test loss 134.8163, time 12.3 sec
epoch 8, train loss 168.9516, test loss 130.0984, time 12.3 sec
epoch 9, train loss 143.4065, test loss 128.6453, time 12.3 sec
epoch 10, train loss 154.1947, test loss 138.1776, time 12.2 sec
epoch 11, train loss 143.6834, test loss 133.0499, time 12.2 sec
epoch 12, train loss 149.8571, test loss 200.6762, time 12.6 sec
epoch 13, train loss 151.6087, test loss 126.7141, time 12.3 sec
epoch 14, train loss 144.7051, test loss 133.1477, time 12.3 sec
epoch 15, train loss 146.6477, test loss 142.6437, time 12.2 sec
epoch 16, train loss 147.951

In [22]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = model2(torch.Tensor(X_test[0:5, :]).to(device))
print(y, Y_train[0:5])

tensor([[1993.2765],
        [1994.4435],
        [1995.2308],
        [1997.6852],
        [1995.0096]], device='cuda:0', grad_fn=<AddmmBackward0>) tensor([2005., 1994., 2007., 2005., 1990.])


# Problema 3

Neste problema, você receberá várias *features* (como altura média, inclinação, etc) descrevendo uma região e o modelo deve predizer qual o tipo da região (floresta, montanha, etc). Mais informações sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/covertype

In [24]:
!wget http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
!gzip covtype.data.gz
data = np.genfromtxt('covtype.data', delimiter=',', dtype=np.float32)

print(data.shape, data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, X[0, :])
print(y.shape, y[0])
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)
train_labels = train_labels - 1
test_labels = test_labels - 1

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-20 01:21:37--  http://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘covtype.data.gz’

covtype.data.gz         [  <=>               ]  10.72M  25.9MB/s    in 0.4s    

2025-05-20 01:21:38 (25.9 MB/s) - ‘covtype.data.gz’ saved [11240707]

gzip: covtype.data.gz already has .gz suffix -- unchanged
(581012, 55) [2.596e+03 5.100e+01 3.000e+00 2.580e+02 0.000e+00 5.100e+02 2.210e+02
 2.320e+02 1.480e+02 6.279e+03 1.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00
 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 0.000e+00 

In [43]:
train_labels[2]

np.float32(0.0)

In [60]:
class NET3 (nn.Module):
  def __init__(self):
    super(NET3, self).__init__()

    self.nn = nn.Sequential(

      nn.Linear(54, 32),
      nn.Dropout(0.5),

      nn.Tanh(),


      nn.Linear(32, 12),
      nn.Dropout(0.3),
      nn.Tanh(),

      nn.Linear(12, 7),
      nn.Dropout(0.15),
      nn.Tanh(),





      nn.Linear(7, 7),

    )

  def forward(self, X):
    out = self.nn(X)

    return out

In [61]:
model3 = NET3().to(device)
loss = nn.CrossEntropyLoss()
opt = optim.Adam(model3.parameters(), lr=0.001, weight_decay = 0.001)

In [62]:
train_validate(net = model3, train_iter = train_iter , test_iter = test_iter, trainer = opt, loss = loss, num_epochs = 20, type='classification')

training on cuda
epoch 1, train loss 1.1998, train acc 0.478, test loss 1.1375, test acc 0.489, time 13.7 sec
epoch 2, train loss 1.0721, train acc 0.508, test loss 0.9784, test acc 0.524, time 12.4 sec
epoch 3, train loss 0.9416, train acc 0.590, test loss 0.9336, test acc 0.575, time 12.3 sec
epoch 4, train loss 0.9309, train acc 0.598, test loss 0.9208, test acc 0.615, time 12.3 sec
epoch 5, train loss 0.9250, train acc 0.601, test loss 0.9178, test acc 0.603, time 12.5 sec
epoch 6, train loss 0.9235, train acc 0.601, test loss 0.9287, test acc 0.612, time 12.5 sec
epoch 7, train loss 0.9194, train acc 0.605, test loss 0.9175, test acc 0.609, time 12.4 sec
epoch 8, train loss 0.9223, train acc 0.604, test loss 0.9542, test acc 0.612, time 17.7 sec
epoch 9, train loss 0.9194, train acc 0.603, test loss 0.9138, test acc 0.617, time 22.6 sec
epoch 10, train loss 0.9184, train acc 0.609, test loss 0.9181, test acc 0.598, time 13.0 sec
epoch 11, train loss 0.9130, train acc 0.610, test l

# Problema 4

Neste problema, você receberá 11 *features* extraídas de tipos de vinhos, e terá que predizer um *score* para cada vinho. Mais sobre esse dataset aqui: https://archive.ics.uci.edu/ml/datasets/Wine+Quality

In [76]:
# download do dataset
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
!wget https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
data_red = np.genfromtxt('winequality-red.csv', delimiter=';', dtype=np.float32, skip_header=1)
data_white = np.genfromtxt('winequality-white.csv', delimiter=';', dtype=np.float32, skip_header=1)
data = np.concatenate((data_red, data_white), axis=0)
data = np.nan_to_num(data)

print(data[0, :])
X, y = data[:, :-1], data[:, -1]
print(X.shape, y.shape)
train_features, test_features, train_labels, test_labels = train_test_split(X, y, test_size=0.33, random_state=42)

batch_size = 100
train_iter = load_array(train_features, train_labels, batch_size)
test_iter = load_array(test_features, test_labels, batch_size, False)

--2025-05-20 01:35:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-red.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-red.csv.3’

winequality-red.csv     [ <=>                ]  82.23K  --.-KB/s    in 0.08s   

2025-05-20 01:35:57 (1.01 MB/s) - ‘winequality-red.csv.3’ saved [84199]

--2025-05-20 01:35:57--  https://archive.ics.uci.edu/ml/machine-learning-databases/wine-quality/winequality-white.csv
Resolving archive.ics.uci.edu (archive.ics.uci.edu)... 128.195.10.252
Connecting to archive.ics.uci.edu (archive.ics.uci.edu)|128.195.10.252|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified
Saving to: ‘winequality-white.csv.3’

winequality-white.c     [ <=>                ] 258.23K  --.-KB/s    in 0.1s    

202

In [69]:
class NET4 (nn.Module):
  def __init__(self):
    super(NET4, self).__init__()

    self.nn = nn.Sequential(

      nn.Linear(11, 4),
      nn.Dropout(0.5),

      nn.ReLU(),


      nn.Linear(4, 12),
      nn.Dropout(0.3),
      nn.ReLU(),

      nn.Linear(12, 24),
      nn.Dropout(0.15),
      nn.ReLU(),





      nn.Linear(24, 1),

    )

  def forward(self, X):
    out = self.nn(X)

    return out

In [70]:
model4 = NET4().to(device)
loss = nn.MSELoss()
opt = optim.Adam(model3.parameters(), lr=0.001, weight_decay = 0.001)

In [75]:
train_validate(net = model4, train_iter = train_iter , test_iter = test_iter, trainer = opt, loss = loss, num_epochs = 50, type='regression')

training on cuda
epoch 1, train loss 28.5391, test loss 28.5342, time 0.2 sec
epoch 2, train loss 28.5459, test loss 28.5342, time 0.2 sec
epoch 3, train loss 28.5306, test loss 28.5342, time 0.2 sec
epoch 4, train loss 28.5182, test loss 28.5342, time 0.2 sec
epoch 5, train loss 28.5558, test loss 28.5342, time 0.2 sec
epoch 6, train loss 28.5343, test loss 28.5342, time 0.2 sec
epoch 7, train loss 28.5754, test loss 28.5342, time 0.2 sec
epoch 8, train loss 28.5218, test loss 28.5342, time 0.2 sec
epoch 9, train loss 28.5623, test loss 28.5342, time 0.1 sec
epoch 10, train loss 28.5697, test loss 28.5342, time 0.1 sec
epoch 11, train loss 28.5251, test loss 28.5342, time 0.1 sec
epoch 12, train loss 28.5536, test loss 28.5342, time 0.1 sec
epoch 13, train loss 28.5474, test loss 28.5342, time 0.1 sec
epoch 14, train loss 28.5377, test loss 28.5342, time 0.1 sec
epoch 15, train loss 28.5559, test loss 28.5342, time 0.1 sec
epoch 16, train loss 28.5839, test loss 28.5342, time 0.1 sec


In [77]:
# mostra o resultado predito para as 5 primeiras instâncias de teste
y = model4(torch.Tensor(test_features[0:5, :]).to(device))
print(y, test_labels[0:5])

tensor([[0.6065],
        [0.5670],
        [0.6725],
        [0.9656],
        [1.0701]], device='cuda:0', grad_fn=<AddmmBackward0>) [8. 5. 7. 6. 6.]
